In [1]:
import requests
import pandas as pd
import urllib3
import time
from datetime import datetime
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Login

In [2]:
def get_with_auth(url, login, password):
    # Define the parameters to send in the GET request
    params = {
        'username': login,
        'password': password
    }

    try:
        # Send GET request with parameters
        response = requests.get(url, params=params, verify=False ) #, cert=("C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/cert.pem", "C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/key.pem"))
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Use response.json() for JSON data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

In [3]:
credentials_full = {
    'Vila Mariana':    {'ip': '10.250.115.39' , 'login': 'WEB', 'password': 'web'},
    'Ibirapuera':      {'ip': '10.250.110.208', 'login': 'WEB', 'password': 'web'},
    'Belo Horizonte':  {'ip': '192.168.215.43', 'login': 'WEB', 'password': 'web'},
    'Brasilia':        {'ip': '192.168.1.20'  , 'login': 'WEB', 'password': 'web'},
    'Salvador':          {'ip': '10.2.0.70'     , 'login': 'WEB', 'password': 'web'},
}

for unidade in credentials_full.keys():
    print(f'staring unidade: {unidade}')
    
    try:
        base_url = f'https://{credentials_full[unidade]["ip"]}:4000/LOGIN'

        content = get_with_auth(base_url, credentials_full[unidade]['login'], credentials_full[unidade]['password'])

        token = content.json()['Token']

        print(f'unidade: {unidade} token: {token} \n')
    except Exception as e:
        print(f'unidade: {unidade} error: {e} \n')


staring unidade: Vila Mariana
unidade: Vila Mariana token: 14AE0575132F46029F38BCD3EA456934 

staring unidade: Ibirapuera
unidade: Ibirapuera token: 86C892ACC7AB4C0F8F5D9DAAE0C6E5B4 

staring unidade: Belo Horizonte
Error fetching URL: HTTPSConnectionPool(host='192.168.215.43', port=4000): Max retries exceeded with url: /LOGIN?username=WEB&password=web (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000017352F58640>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
unidade: Belo Horizonte error: 'NoneType' object has no attribute 'json' 

staring unidade: Brasilia
unidade: Brasilia token: 92054045370E4B6684850A25D37B844F 

staring unidade: Salvador
Error fetching URL: HTTPSConnectionPool(host='10.2.0.70', port=4000): Max retries exceeded with url: /LOGIN?username=WE

In [8]:
base_url = 'https://10.250.110.208:4000/'
    
login_url = "LOGIN"

URL = base_url + login_url

content = get_with_auth(URL, 'Web', 'web')

token = content.json()['Token']

token

'6DDD2A2D96B44601A53FA313BDB73EE5'

## Patients

In [5]:
def get_patients(base_url, token):
    # Define the URL for getting patients
    url = base_url + "/GET/patients"

    # Define the headers with the API token
    headers = {
        'API-token': token
    }

    try:
        # Send GET request to fetch patients
        response = requests.get(url, headers=headers, verify=False)
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"Error fetching patients: {e}")
        return None

url = base_url + "/GET/patients"


response = get_patients(base_url, token)

if response is not None:
    # Convert the response to a DataFrame
    patients = response.json()
    print(patients)
    df_patients = pd.DataFrame(patients['Patients'])



{'Patients': [{'PatientIDx': 'NEXTGEN_43895.4511009838', 'PatientID': '123', 'FirstName': 'katia'}, {'PatientIDx': 'S4JP138913238_43895.4537480671', 'PatientID': '155', 'FirstName': 'Patent', 'LastName': 'name'}, {'PatientIDx': 'NEXTGEN_43895.6049278935', 'PatientID': 'overnight test', 'FirstName': 'overnight', 'LastName': 'test'}, {'PatientIDx': 'NEXTGEN_43899.5469196528', 'PatientID': '1106', 'FirstName': 'karina'}, {'PatientIDx': 'NEXTGEN_43899.5580717824', 'PatientID': '0806', 'FirstName': 'Pamela'}, {'PatientIDx': 'NEXTGEN_43899.5783226042', 'PatientID': '2106', 'FirstName': 'Ana', 'LastName': 'Paula'}, {'PatientIDx': 'S4JP138957021_43902.6256269676', 'PatientID': '000000001', 'FirstName': 'REIS,ANA', 'LastName': '21/06/1982', 'DateOfBirth': '1982.06.01'}, {'PatientIDx': 'S4JP138957021_43902.6461418519', 'PatientID': '00000008', 'FirstName': 'SANTOS, PAMELA', 'LastName': 'O.', 'DateOfBirth': '1993.06.01'}, {'PatientIDx': 'S4JP138957021_43903.5718785185', 'PatientID': '1406', 'Firs

In [8]:
df_patients.head()

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
0,NEXTGEN_43622.7870662732,1111,first,test,NaN
1,PC10T4L72760_43623.4196205208,9999,Jane,Doe,NaN
2,NEXTGEN_43622.6655321528,0000,cam,test,NaN
3,PC10T4L77647_43623.5757282639,send mail to Martin,End,Tuesday,NaN
4,PC10T4L790165_43625.4810629630,62.399,Amanda Alves,Pollone,1983.12.01


# Treatments

In [9]:
def get_treatments(base_url, token, patientIDx):
    """
    Retrieve treatments for a given patientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing treatments, or None if request fails.
    """
    url = f"{base_url}/GET/TREATMENT"
    params = {'patientIDx': patientIDx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching treatments: {e}")
        return None

In [10]:
patient_idx = 'PC1R85KM_45801.4523230440'
treatment_number = get_treatments(base_url, token, patient_idx)
print(treatment_number)
treatment_number = treatment_number['TreatmentList'][0]
treatment_number

{'TreatmentList': ['2025  - 881']}


'2025  - 881'

# Embryo ID

In [11]:
def get_embryo_id(base_url, token, patientIDx, treatment_name):
    """
    Retrieve embryo ID data for a given patientIDx and treatment name from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo ID data, or None if request fails.
    """
    url = f"{base_url}/GET/embryoID"
    params = {
        'PatientIDx': patientIDx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo ID: {e}")
        return None

In [12]:
get_embryo_id(base_url, token, patient_idx, treatment_number)

{'EmbryoIDList': ['D2025.05.24_S03537_I3253_P-1',
  'D2025.05.24_S03537_I3253_P-2',
  'D2025.05.24_S03537_I3253_P-3']}

In [13]:
embryo_idx = get_embryo_id(base_url, token, patient_idx, treatment_number)['EmbryoIDList'][0]
embryo_idx

'D2025.05.24_S03537_I3253_P-1'

# Fertilization time

In [14]:
def get_fertilization_time(base_url, token, embryo_id):
    """
    Retrieve fertilization time for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing fertilization time, or None if request fails.
    """
    url = f"{base_url}/GET/fertilizationtime"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fertilization time: {e}")
        return None


In [15]:
# Example usage:
get_fertilization_time(base_url, token, embryo_idx)

{'FertilizationTime': '2025.05.24 11:50:00'}

# Image runs

In [16]:
def get_image_runs(base_url, token, embryo_id):
    """
    Retrieve image runs for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing image runs, or None if request fails.
    """
    url = f"{base_url}/GET/imageruns"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image runs: {e}")
        return None


In [17]:

# Example usage:
get_image_runs(base_url, token, embryo_idx)

{'ImageRuns': [{'Run': 1, 'Time': 0.2},
  {'Run': 2, 'Time': 0.3},
  {'Run': 3, 'Time': 0.4},
  {'Run': 4, 'Time': 0.6},
  {'Run': 5, 'Time': 0.8},
  {'Run': 6, 'Time': 1},
  {'Run': 7, 'Time': 1.2},
  {'Run': 8, 'Time': 1.4},
  {'Run': 9, 'Time': 1.5},
  {'Run': 10, 'Time': 1.7},
  {'Run': 11, 'Time': 1.9},
  {'Run': 12, 'Time': 2.1},
  {'Run': 13, 'Time': 2.3},
  {'Run': 14, 'Time': 2.4},
  {'Run': 15, 'Time': 2.6},
  {'Run': 16, 'Time': 2.8},
  {'Run': 17, 'Time': 3},
  {'Run': 18, 'Time': 3.2},
  {'Run': 19, 'Time': 3.3},
  {'Run': 20, 'Time': 3.5},
  {'Run': 21, 'Time': 3.7},
  {'Run': 22, 'Time': 3.9},
  {'Run': 23, 'Time': 4.1},
  {'Run': 24, 'Time': 4.3},
  {'Run': 25, 'Time': 4.4},
  {'Run': 26, 'Time': 4.6},
  {'Run': 27, 'Time': 4.8},
  {'Run': 28, 'Time': 5},
  {'Run': 29, 'Time': 5.2},
  {'Run': 30, 'Time': 5.3},
  {'Run': 31, 'Time': 5.5},
  {'Run': 32, 'Time': 5.7},
  {'Run': 33, 'Time': 5.9},
  {'Run': 34, 'Time': 6.1},
  {'Run': 35, 'Time': 6.2},
  {'Run': 36, 'Time': 

# Evaluation - KID score

In [18]:
def get_evaluation(base_url, token, embryo_id):
    """
    Retrieve evaluation data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing evaluation data, or None if request fails.
    """
    url = f"{base_url}/GET/evaluation"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching evaluation: {e}")
        return None

In [19]:
get_evaluation(base_url, token, embryo_idx)

{'Model': 'KIDScoreD5 v3.3',
 'Evaluation': '1.5',
 'User': 'ADMIN',
 'EvaluationDate': '2025.05.29'}

# Emrbyo Fate

In [20]:
def get_embryo_fate(base_url, token, embryo_id):
    """
    Retrieve embryo fate data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo fate data, or None if request fails.
    """
    url = f"{base_url}/GET/embryofate"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo fate: {e}")
        return None


In [21]:

# Example usage:
get_embryo_fate(base_url, token, embryo_idx)

{'EmbryoFate': 'Freeze'}

# Embryo Details

In [22]:
def get_embryo_details(base_url, token, embryo_id):
    """
    Retrieve detailed embryo information for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo details, or None if request fails.
    """
    url = f"{base_url}/GET/embryodetails"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo details: {e}")
        return None


In [23]:

# Example usage:
get_embryo_details(base_url, token, embryo_idx)

{'InstrumentNumber': 3253,
 'Position': 6,
 'WellNumber': 1,
 'FertilizationTime': '2025.05.24 11:50:00',
 'EmbryoFate': 'Freeze',
 'Description': 'ALTERADO - MONOSSOMIA PARCIAL chr 8p  XY',
 'EmbryoDescriptionID': 'AA1'}

# Embryo Data

In [24]:
def get_embryo_data(base_url, token, patient_idx, treatment_name):
    """
    Retrieve embryo data for a given PatientIDx and TreatmentName from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo data, or None if request fails.
    """
    url = f"{base_url}/GET/embryodata"
    params = {
        'PatientIDx': patient_idx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo data: {e}")
        return None

In [ ]:



get_embryo_data(base_url, token, patient_idx, treatment_number)

{'EmbryoDataList': [{'EmbryoID': 'D2025.05.24_S03537_I3253_P-1',
   'EmbryoDetails': {'InstrumentNumber': 3253,
    'Position': 6,
    'WellNumber': 1,
    'FertilizationTime': '2025.05.24 11:50:00',
    'EmbryoFate': 'Freeze',
    'Description': 'ALTERADO - MONOSSOMIA PARCIAL chr 8p  XY',
    'EmbryoDescriptionID': 'AA1'},
   'AnnotationList': [{'Name': 'PN',
     'Time': 18,
     'Value': '2',
     'Timestamp': '2025.05.25 05:47:47'},
    {'Name': 'tPNf',
     'Time': 26.5,
     'Value': '45802.5966041782',
     'Timestamp': '2025.05.25 14:19:06'},
    {'Name': 't2',
     'Time': 29,
     'Value': '45802.7017435764',
     'Timestamp': '2025.05.25 16:50:30'},
    {'Name': 'FRAG2CAT',
     'Time': 32.3,
     'Value': '0-10%',
     'Timestamp': '2025.05.25 20:05:09'},
    {'Name': 'Nuclei2',
     'Time': 35,
     'Value': '2',
     'Timestamp': '2025.05.25 22:47:22'},
    {'Name': 'MN2Type',
     'Time': 35,
     'Value': 'Multi',
     'Timestamp': '2025.05.25 22:47:22'},
    {'Name': '

# Transfer

In [26]:
def get_transfers(base_url, token, patient_idx):
    """
    Retrieve transfer data for a given PatientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing transfer data, or None if request fails.
    """
    url = f"{base_url}/GET/transfers"
    params = {'PatientIDx': patient_idx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching transfers: {e}")
        return None


In [27]:

# Example usage:
get_transfers(base_url, token, patient_idx)

Error fetching transfers: Expecting value: line 1 column 1 (char 0)


# IDA Score

In [28]:
def get_idascore(base_url, token):
    """
    Retrieve IDASCORE data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing IDASCORE data, or None if request fails.
    """
    url = f"{base_url}/GET/IDASCORE"
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching IDASCORE: {e}")
        return None

In [29]:
# Call the function to get IDASCORE data
idascore_data = get_idascore(base_url, token)

# Check if data was retrieved successfully and 'Scores' key exists
if idascore_data and 'Scores' in idascore_data:
    # Convert the 'Scores' list into a DataFrame
    df_idascore = pd.DataFrame(idascore_data['Scores'])
else:
    print("No IDASCORE data found or 'Scores' key missing in the response.")
    df_idascore = pd.DataFrame() # Create an empty DataFrame if no data

df_idascore.head()

,EmbryoID,Viability,Time,Version,Timestamp
0,D2019.06.19_S00028_I3253_P-1,1.03,115,2.0.4,2024.04.30 07:55:37
1,D2019.06.19_S00028_I3253_P-2,1.05,115,2.0.4,2024.04.30 07:55:35
2,D2019.06.19_S00028_I3253_P-3,1.06,115,2.0.4,2024.04.30 07:55:33
3,D2019.06.19_S00028_I3253_P-4,1.01,115,2.0.4,2024.04.30 07:56:06
4,D2019.07.03_S00054_I3253_P-2,3.93,120,2.0.4,2024.05.28 15:20:00


# Fluxo de um embryoscope

In [30]:
def rate_limited_request(func, *args, **kwargs):
    """Execute function with rate limiting (10 requests per second)"""
    time.sleep(0.1)  # 100ms delay to stay under 10 requests/second
    return func(*args, **kwargs)

def get_all_treatments_for_patients(df_patients, base_url, token):
    """Get all treatments for all patients"""
    all_treatments = []
    
    print(f"Processing {len(df_patients)} patients...")
    
    for idx, patient in df_patients.iterrows():
        patient_idx = patient['PatientIDx']
        print(f"Processing patient {idx+1}/{len(df_patients)}: {patient_idx}")
        
        # Get treatments with rate limiting
        treatments_data = rate_limited_request(get_treatments, base_url, token, patient_idx)

        if treatments_data and 'TreatmentList' in treatments_data:
            for treatment_name in treatments_data['TreatmentList']:
                # Create treatment record with patient reference
                treatment_record = {
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name
                }
                all_treatments.append(treatment_record)
    
    df_treatments = pd.DataFrame(all_treatments)
    print(f"Found {len(df_treatments)} treatments across all patients")
    return df_treatments

def get_all_embryo_data(df_treatments, base_url, token):
    """Get embryo data for all treatments and break down nested dictionaries into columns"""
    all_embryo_data = []
    
    print(f"Processing {len(df_treatments)} treatments...")
    
    for idx, treatment in df_treatments.iterrows():
        patient_idx = treatment['PatientIDx']
        treatment_name = treatment['TreatmentName']
        
        print(f"Processing treatment {idx+1}/{len(df_treatments)}: {treatment_name}")
        
        # Get embryo data with rate limiting
        embryo_data = rate_limited_request(get_embryo_data, base_url, token, patient_idx, treatment_name)
        
        if embryo_data and 'EmbryoDataList' in embryo_data:
            for embryo in embryo_data['EmbryoDataList']:
                # Start with basic embryo info
                embryo_record = {
                    'EmbryoID': embryo['EmbryoID'],
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name,
                    'AnnotationList': embryo.get('AnnotationList')
                }
                
                # Break down EmbryoDetails into separate columns
                if 'EmbryoDetails' in embryo and embryo['EmbryoDetails']:
                    details = embryo['EmbryoDetails']
                    embryo_record.update({
                        'InstrumentNumber': details.get('InstrumentNumber'),
                        'Position': details.get('Position'),
                        'WellNumber': details.get('WellNumber'),
                        'FertilizationTime': details.get('FertilizationTime'),
                        'EmbryoFate': details.get('EmbryoFate'),
                        'Description': details.get('Description'),
                        'EmbryoDescriptionID': details.get('EmbryoDescriptionID')
                    })
                
                # Break down Evaluation into separate columns
                if 'Evaluation' in embryo and embryo['Evaluation']:
                    evaluation = embryo['Evaluation']
                    embryo_record.update({
                        'EvaluationModel': evaluation.get('Model'),
                        'EvaluationScore': evaluation.get('Evaluation'),
                        'EvaluationUser': evaluation.get('User'),
                        'EvaluationDate': evaluation.get('EvaluationDate')
                    })
                
                all_embryo_data.append(embryo_record)
    
    df_embryo_data = pd.DataFrame(all_embryo_data)
    print(f"Found {len(df_embryo_data)} embryos across all treatments")
    return df_embryo_data


In [31]:
# Execute the flux
print("=== Starting Data Collection Flux ===")
print(f"Start time: {datetime.now()}")

# Step 1: We already have patients in df_patients
print(f"\nStep 1: Patients loaded - {len(df_patients)} patients")

df_patients.tail()

=== Starting Data Collection Flux ===
Start time: 2025-07-10 18:25:07.796357

Step 1: Patients loaded - 2644 patients


,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
2639,PC1R85KM_45847.4244880093,178.799,"REZENDE, FERNANDA SIMOES PIRES COSTA",02/03/1988,1988.03.02
2640,PC1R85KM_45848.5024507176,157.186,"AZEVEDO, FLAVIA DE M.",27/12/1979,1979.12.27
2641,PC1R85KM_45775.3131217824,883.218,"PEDRO, SHEILA",01/08/1980,1980.08.01
2642,PC1R85KM_45833.3483204745,841.319,"RODRIGUES, DANIELA S.",23/06/1980,1980.06.23
2643,PC1R85KM_45848.7638946759,873.998,"BENETI, GISELLE M.",15/10/1982,1982.10.15


In [32]:

# Step 2: Get all treatments
print("\nStep 2: Collecting treatments...")
df_treatments = get_all_treatments_for_patients(df_patients, base_url, token)

df_treatments.tail()


Step 2: Collecting treatments...
Processing 2644 patients...
Processing patient 1/2644: NEXTGEN_43622.7870662732
Processing patient 2/2644: PC10T4L72760_43623.4196205208
Processing patient 3/2644: NEXTGEN_43622.6655321528
Processing patient 4/2644: PC10T4L77647_43623.5757282639
Processing patient 5/2644: PC10T4L790165_43625.4810629630
Processing patient 6/2644: PC10T4L741253_43626.4449943171
Processing patient 7/2644: PC10T4L741253_43626.4518250926
Processing patient 8/2644: PC10T4L790165_43624.3678845139
Processing patient 9/2644: PC10T4L797965_43627.5657467361
Processing patient 10/2644: PC10T4L769663_43628.4976527894
Processing patient 11/2644: PC10T4L797614_43631.4652546296
Processing patient 12/2644: PC10T4L77243_43629.5771579398
Processing patient 13/2644: PC10T4L799281_43634.6290361111
Processing patient 14/2644: PC10T4L720650_43635.4819136343
Processing patient 15/2644: PC10T4L720650_43640.5834488773
Processing patient 16/2644: PC10T4L720650_43636.4914582523
Processing patient

,PatientIDx,TreatmentName
3511,PC1R85KM_45848.5024507176,2025 - 1184
3512,PC1R85KM_45775.3131217824,2025-697
3513,PC1R85KM_45775.3131217824,2025 - 1101
3514,PC1R85KM_45833.3483204745,2025 - 1095
3515,PC1R85KM_45848.7638946759,2025 - 1191


In [33]:

# Step 3: Get all embryo data
print("\nStep 3: Collecting embryo data...")
df_embryo_data = get_all_embryo_data(df_treatments, base_url, token)

df_embryo_data.tail()


Step 3: Collecting embryo data...
Processing 3516 treatments...
Processing treatment 1/3516: 1icsi
Processing treatment 2/3516: 2022 - 12
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 3/3516: Test August
Processing treatment 4/3516: 1
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 5/3516: 2
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 6/3516: 1
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 7/3516: 1
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 8/3516: 2
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 9/3516: 3
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 10/3516: 1
Processing treatment 11/3516: 05.06.2019
Processing treatment 12/3516: 05.06.2019
Processing treatment 13/3516: 

,EmbryoID,PatientIDx,TreatmentName,AnnotationList,InstrumentNumber,Position,WellNumber,FertilizationTime,EmbryoFate,Description,EmbryoDescriptionID,EvaluationModel,EvaluationScore,EvaluationUser,EvaluationDate
30561,D2025.07.10_S03628_I3253_P-2,PC1R85KM_45848.7638946759,2025 - 1191,None,3253,12,2,2025.07.10 17:00:00,Unknown,None,AA2,NaN,NaN,NaN,NaN
30562,D2025.07.10_S03628_I3253_P-3,PC1R85KM_45848.7638946759,2025 - 1191,None,3253,12,3,2025.07.10 17:00:00,Unknown,None,AA3,NaN,NaN,NaN,NaN
30563,D2025.07.10_S03628_I3253_P-4,PC1R85KM_45848.7638946759,2025 - 1191,None,3253,12,4,2025.07.10 17:00:00,Unknown,None,AA4,NaN,NaN,NaN,NaN
30564,D2025.07.10_S03628_I3253_P-5,PC1R85KM_45848.7638946759,2025 - 1191,None,3253,12,5,2025.07.10 17:00:00,Unknown,None,AA5,NaN,NaN,NaN,NaN
30565,D2025.07.10_S03628_I3253_P-6,PC1R85KM_45848.7638946759,2025 - 1191,None,3253,12,6,2025.07.10 17:00:00,Unknown,None,AA6,NaN,NaN,NaN,NaN


In [34]:

print(f"\n=== Data Collection Complete ===")
print(f"End time: {datetime.now()}")
print(f"Total patients: {len(df_patients)}")
print(f"Total treatments: {len(df_treatments)}")
print(f"Total embryos: {len(df_embryo_data)}")


=== Data Collection Complete ===
End time: 2025-07-10 19:11:07.530737
Total patients: 2644
Total treatments: 3516
Total embryos: 30566


In [35]:
# Merge df_patients with df_treatments on 'PatientIDx'
df_merged = pd.merge(df_patients, df_treatments, on='PatientIDx', how='left')

# Merge the result with df_embryo_data on 'PatientIDx' and 'TreatmentName'
df_final_merged = pd.merge(df_merged, df_embryo_data, on=['PatientIDx', 'TreatmentName'], how='left')

# Merge df_final_merged with df_idascore on 'EmbryoID'
df_final = pd.merge(df_final_merged, df_idascore, on='EmbryoID', how='inner')

print("Shape of df_patients:", df_patients.shape)
print("Shape of df_treatments:", df_treatments.shape)
print("Shape of df_embryo_data:", df_embryo_data.shape)
print("Shape of df_final_merged:", df_final_merged.shape)
print("Shape of df_idascore:", df_idascore.shape)
print("Shape of df_final:", df_final.shape)

df_final.tail()

Shape of df_patients: (2644, 5)
Shape of df_treatments: (3516, 2)
Shape of df_embryo_data: (30566, 15)
Shape of df_final_merged: (30598, 19)
Shape of df_idascore: (9438, 5)
Shape of df_final: (9438, 23)


,PatientIDx,PatientID,FirstName,LastName,DateOfBirth,TreatmentName,EmbryoID,AnnotationList,InstrumentNumber,Position,...,Description,EmbryoDescriptionID,EvaluationModel,EvaluationScore,EvaluationUser,EvaluationDate,Viability,Time,Version,Timestamp
9433,PC1R85KM_45814.5561627546,874.641,"VOLKMER, STEPHANIE CHRIST",16/03/1991,1991.03.16,2025 - 975,D2025.06.06_S00860_I4268_8-6,"[{'Name': 'PN', 'Time': 18.6, 'Value': '2', 'T...",4268.0,5.0,...,None,AA6,KIDScoreD5 v3.3,1.5,ADMIN,2025.06.12,1.13,145,2.0.5,2025.06.12 14:46:27
9434,PC1R85KM_45814.5561627546,874.641,"VOLKMER, STEPHANIE CHRIST",16/03/1991,1991.03.16,2025 - 975,D2025.06.06_S00860_I4268_8-7,"[{'Name': 'PN', 'Time': 18.6, 'Value': '2', 'T...",4268.0,5.0,...,ANEUPLOIDE -22 XY,AA7,KIDScoreD5 v3.3,8.9,ADMIN,2025.06.12,5.16,145,2.0.5,2025.06.12 14:46:28
9435,PC1R85KM_45814.5561627546,874.641,"VOLKMER, STEPHANIE CHRIST",16/03/1991,1991.03.16,2025 - 975,D2025.06.06_S00860_I4268_8-8,"[{'Name': 'PN', 'Time': 18.6, 'Value': '0', 'T...",4268.0,5.0,...,None,AA8,KIDScoreD5 v3.3,0,ADMIN,2025.06.12,1.01,145,2.0.5,2025.06.12 14:46:27
9436,PC1R85KM_45814.5561627546,874.641,"VOLKMER, STEPHANIE CHRIST",16/03/1991,1991.03.16,2025 - 975,D2025.06.06_S00860_I4268_8-9,"[{'Name': 'PN', 'Time': 18.6, 'Value': '2', 'T...",4268.0,5.0,...,EUPLOIDE XY,AA9,KIDScoreD5 v3.3,8.1,ADMIN,2025.06.12,4.37,145,2.0.5,2025.06.12 14:46:28
9437,PC1R85KM_45814.5561627546,874.641,"VOLKMER, STEPHANIE CHRIST",16/03/1991,1991.03.16,2025 - 975,D2025.06.06_S00860_I4268_8-10,"[{'Name': 'PN', 'Time': 18.6, 'Value': '2', 'T...",4268.0,5.0,...,EUPLOIDE XX,AA10,KIDScoreD5 v3.3,7.9,ADMIN,2025.06.12,4.36,145,2.0.5,2025.06.12 14:46:28


In [36]:
# Create data_output folder if it doesn't exist
os.makedirs('data_output', exist_ok=True)

# Save dataframes to CSV files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save without timestamp
df_patients.to_csv('data_output/patients.csv', index=False, quoting=1)
df_treatments.to_csv('data_output/treatments.csv', index=False, quoting=1)
df_embryo_data.to_csv('data_output/embryo_data.csv', index=False, quoting=1)
df_final.to_csv('data_output/final_merged_with_idascore.csv', index=False, quoting=1)

# Save with timestamp
df_patients.to_csv(f'data_output/patients_{timestamp}.csv', index=False, quoting=1)
df_treatments.to_csv(f'data_output/treatments_{timestamp}.csv', index=False, quoting=1)
df_embryo_data.to_csv(f'data_output/embryo_data_{timestamp}.csv', index=False, quoting=1)
df_final.to_csv(f'data_output/final_merged_with_idascore_{timestamp}.csv', index=False, quoting=1)

print(f"Files saved to data_output folder with timestamp: {timestamp}")

Files saved to data_output folder with timestamp: 20250710_191107


In [37]:
# 1. Patients with no treatment
# Get unique PatientIDx from df_treatments (patients who have at least one treatment)
patients_with_treatment_ids = df_treatments['PatientIDx'].unique()
# Filter df_patients to find those whose PatientIDx is not in patients_with_treatment_ids
patients_without_treatment_df = df_patients[~df_patients['PatientIDx'].isin(patients_with_treatment_ids)]
num_patients_no_treatment = patients_without_treatment_df.shape[0]

# 2. Treatments with no embryo
# Create a marker for treatments that have at least one embryo
# Get unique (PatientIDx, TreatmentName) combinations from df_embryo_data
unique_treatments_in_embryo_data = df_embryo_data[['PatientIDx', 'TreatmentName']].drop_duplicates()
unique_treatments_in_embryo_data['has_embryo_marker'] = True

# Left merge df_treatments with the marked treatments from df_embryo_data
merged_treatments_check = pd.merge(
    df_treatments,
    unique_treatments_in_embryo_data,
    on=['PatientIDx', 'TreatmentName'],
    how='left'
)
# Treatments with no embryo will have NaN in 'has_embryo_marker'
num_treatments_no_embryo = merged_treatments_check['has_embryo_marker'].isnull().sum()

# 3. Embryos with no EvaluationScore AND no Viability
# Left merge df_embryo_data with selected columns from df_idascore to get Viability for each embryo
# This ensures all embryos from df_embryo_data are included
merged_embryos_for_score_check = pd.merge(
    df_embryo_data,
    df_idascore[['EmbryoID', 'Viability']],
    on='EmbryoID',
    how='left'
)

# Condition for missing EvaluationScore: either NaN/None or the string 'NA'
is_eval_score_missing = merged_embryos_for_score_check['EvaluationScore'].isnull() | \
                        (merged_embryos_for_score_check['EvaluationScore'] == 'NA')

# Condition for missing Viability: NaN (will occur if an embryo is not in df_idascore or if Viability was NaN there)
is_viability_missing = merged_embryos_for_score_check['Viability'].isnull()

# Filter for embryos missing both
embryos_missing_both_scores = merged_embryos_for_score_check[is_eval_score_missing & is_viability_missing]
num_embryos_no_eval_no_viability = embryos_missing_both_scores.shape[0]

# Print the report
print("Data Quality Report:")
print(f"- Patients with no treatment: {num_patients_no_treatment}")
print(f"- Treatments with no embryo: {num_treatments_no_embryo}")
print(f"- Embryos with no EvaluationScore AND no Viability: {num_embryos_no_eval_no_viability}")

Data Quality Report:
- Patients with no treatment: 1
- Treatments with no embryo: 31
- Embryos with no EvaluationScore AND no Viability: 14156


In [38]:
df_final.columns

Index(['PatientIDx', 'PatientID', 'FirstName', 'LastName', 'DateOfBirth',
       'TreatmentName', 'EmbryoID', 'AnnotationList', 'InstrumentNumber',
       'Position', 'WellNumber', 'FertilizationTime', 'EmbryoFate',
       'Description', 'EmbryoDescriptionID', 'EvaluationModel',
       'EvaluationScore', 'EvaluationUser', 'EvaluationDate', 'Viability',
       'Time', 'Version', 'Timestamp'],
      dtype='object')

In [39]:
df_final['EmbryoFate'].value_counts(dropna=False)

EmbryoFate
Avoid                   4455
Freeze                  4287
Unknown                  365
Transfer                 305
Undecided                 15
FrozenEmbryoTransfer      11
Name: count, dtype: int64